In [2]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [2]:
results = requests.get("https://api.nytimes.com/svc/books/v3/lists/2023-05-26/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [3]:
results.status_code

200

In [15]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "combined-print-and-e-book-fiction"   # you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "keys.json"

bestsellers = []
weeks = []
num_calls_to_make = 1  # Number of API calls to make
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date.today() - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s?list=%s&published-date=%s&api-key=%s" % (BOOKS_ROOT, LIST, curr_date, API_KEY)
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            for result in results['results']:
                    for book_details in result['book_details']:
                        book_info = {
                            'title': book_details['title'],
                            'description': book_details['description'],
                            'contributor': book_details['contributor'],
                            'author': book_details['author'],
                            'contributor_note': book_details['contributor_note'],
                            'price': book_details['price'],
                            'age_group': book_details['age_group'],
                            'publisher': book_details['publisher'],
                            'primary_isbn13': book_details['primary_isbn13'],
                            'primary_isbn10': book_details['primary_isbn10'],
                            'rank': result['rank'],
                            'rank_last_week': result['rank_last_week'],
                            'weeks_on_list': result['weeks_on_list'],
                            'asterisk': result['asterisk'],
                            'dagger': result['dagger']
                        }
                        bestsellers.append(book_info)
                        weeks.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)
  

2023-05-26
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2023-05-26&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-05-19
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2023-05-19&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-05-12
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2023-05-12&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-05-05
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2023-05-05&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-04-28
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2023-04-28&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-04-21
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2023-04-21&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-04-14

In [16]:
combined_print_and_e_book_fiction = pd.json_normalize(bestsellers)

In [17]:
combined_print_and_e_book_fiction.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,HAPPY PLACE,A former couple pretend to be together for the...,by Emily Henry,Emily Henry,,0.00,,Berkley,9780593441275,0593441273,1,2,3,0,0
1,IT ENDS WITH US,A battered wife raised in a violent home attem...,by Colleen Hoover,Colleen Hoover,,0.00,,Atria,9781501110368,1501110365,2,7,100,0,0
2,THE MAKING OF ANOTHER MAJOR MOTION PICTURE MAS...,A World War II soldier’s experiences inspire a...,by Tom Hanks,Tom Hanks,,0.00,,Knopf,9780525655596,052565559X,3,0,1,0,0
3,FOURTH WING,Violet Sorrengail is urged by the commanding g...,by Rebecca Yarros,Rebecca Yarros,,0.00,,Red Tower,9781649374042,1649374046,4,5,2,0,0
4,LESSONS IN CHEMISTRY,A scientist and single mother living in Califo...,by Bonnie Garmus,Bonnie Garmus,,0.00,,Doubleday,9780385547345,038554734X,5,8,27,0,0


In [18]:
combined_print_and_e_book_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
3625,ORIGIN,After reconnecting with one of his first stude...,by Dan Brown,Dan Brown,,0.00,,Doubleday,9780385542692,0385542690,11,3,31,0,0
3626,ELEANOR OLIPHANT IS COMPLETELY FINE,A young woman’s well-ordered life is disrupted...,by Gail Honeyman,Gail Honeyman,,0.00,,Penguin,9780735220690,0735220697,12,0,15,0,0
3627,CHINA RICH GIRLFRIEND,The second book in the Crazy Rich Asians trilo...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780804172066,0804172064,13,9,8,0,0
3628,CHRISTMAS CAKE MURDER,"While on the verge of opening her bakery, Hann...",by Joanne Fluke,Joanne Fluke,,0.00,,Kensington,9781617732331,1617732338,14,0,1,0,0
3629,RICH PEOPLE PROBLEMS,The final book in the Crazy Rich Asians trilog...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780525432371,052543237X,15,15,7,0,0


In [19]:
len(combined_print_and_e_book_fiction)

3630

In [20]:
len(weeks)

3630

In [21]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
combined_print_and_e_book_fiction['rank'] = combined_print_and_e_book_fiction['rank'].astype(str)
filter_15 = combined_print_and_e_book_fiction['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [22]:

combined_print_and_e_book_fiction = combined_print_and_e_book_fiction[filter_15]

In [23]:
len(combined_print_and_e_book_fiction)

3630

In [24]:
len(weeks)

3630

In [25]:
#add a week column
combined_print_and_e_book_fiction['week'] = weeks

#and check to make sure it worked correctly
combined_print_and_e_book_fiction.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
3610,BLOOD COMMUNION,The latest in the Vampire Chronicles series de...,by Anne Rice,Anne Rice,,0.00,,Knopf,9781524732646,1524732648,11,0,1,0,0,2018-10-19
3611,ELEANOR OLIPHANT IS COMPLETELY FINE,A young woman’s well-ordered life is disrupted...,by Gail Honeyman,Gail Honeyman,,0.00,,Penguin,9780735220690,0735220697,12,12,16,0,0,2018-10-19
3612,WAR OF THE WOLF,The Saxon Tales series continues as Uhtred of ...,by Bernard Cornwell,Bernard Cornwell,,0.00,,Harper,9780062563194,006256319X,13,0,1,0,0,2018-10-19
3613,AN ABSOLUTELY REMARKABLE THING,"After posting a video that goes viral, April M...",by Hank Green,Hank Green,,0.00,,Dutton,9781524743444,1524743445,14,2,2,0,0,2018-10-19
3614,CHINA RICH GIRLFRIEND,The second book in the Crazy Rich Asians trilo...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780804172066,0804172064,15,13,9,0,0,2018-10-19
3615,VINCE FLYNN: RED WAR,When the Russian prime minister plots to invad...,by Kyle Mills,Kyle Mills,,0.00,,Emily Bestler/Atria,9781501190612,150119061X,1,0,1,0,0,2018-10-12
3616,AN ABSOLUTELY REMARKABLE THING,"After posting a video that goes viral, April M...",by Hank Green,Hank Green,,0.00,,Dutton,9781524743444,1524743445,2,0,1,0,0,2018-10-12
3617,TRANSCRIPTION,"Ten years later, figures from a BBC radio prod...",by Kate Atkinson,Kate Atkinson,,0.00,,"Little, Brown",9780316479752,0316479756,3,0,1,0,0,2018-10-12
3618,CRAZY RICH ASIANS,A New Yorker gets a surprise when she spends t...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780345803788,0345803787,4,5,16,0,0,2018-10-12
3619,LETHAL WHITE,The fourth book in the Cormoran Strike series....,by Robert Galbraith,Robert Galbraith,,0.00,,"Mulholland/Little, Brown",9780316422741,0316422746,5,1,2,0,0,2018-10-12


In [28]:
#and a column indicating which list these come from
k2 = 3630
weekly_fiction_list = ['Combined Print & E-Book Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3630

In [29]:
#add nyt_list as a column
combined_print_and_e_book_fiction['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
combined_print_and_e_book_fiction.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
3610,BLOOD COMMUNION,The latest in the Vampire Chronicles series de...,by Anne Rice,Anne Rice,,0.00,,Knopf,9781524732646,1524732648,11,0,1,0,0,2018-10-19,Combined Print & E-Book Fiction
3611,ELEANOR OLIPHANT IS COMPLETELY FINE,A young woman’s well-ordered life is disrupted...,by Gail Honeyman,Gail Honeyman,,0.00,,Penguin,9780735220690,0735220697,12,12,16,0,0,2018-10-19,Combined Print & E-Book Fiction
3612,WAR OF THE WOLF,The Saxon Tales series continues as Uhtred of ...,by Bernard Cornwell,Bernard Cornwell,,0.00,,Harper,9780062563194,006256319X,13,0,1,0,0,2018-10-19,Combined Print & E-Book Fiction
3613,AN ABSOLUTELY REMARKABLE THING,"After posting a video that goes viral, April M...",by Hank Green,Hank Green,,0.00,,Dutton,9781524743444,1524743445,14,2,2,0,0,2018-10-19,Combined Print & E-Book Fiction
3614,CHINA RICH GIRLFRIEND,The second book in the Crazy Rich Asians trilo...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780804172066,0804172064,15,13,9,0,0,2018-10-19,Combined Print & E-Book Fiction
3615,VINCE FLYNN: RED WAR,When the Russian prime minister plots to invad...,by Kyle Mills,Kyle Mills,,0.00,,Emily Bestler/Atria,9781501190612,150119061X,1,0,1,0,0,2018-10-12,Combined Print & E-Book Fiction
3616,AN ABSOLUTELY REMARKABLE THING,"After posting a video that goes viral, April M...",by Hank Green,Hank Green,,0.00,,Dutton,9781524743444,1524743445,2,0,1,0,0,2018-10-12,Combined Print & E-Book Fiction
3617,TRANSCRIPTION,"Ten years later, figures from a BBC radio prod...",by Kate Atkinson,Kate Atkinson,,0.00,,"Little, Brown",9780316479752,0316479756,3,0,1,0,0,2018-10-12,Combined Print & E-Book Fiction
3618,CRAZY RICH ASIANS,A New Yorker gets a surprise when she spends t...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780345803788,0345803787,4,5,16,0,0,2018-10-12,Combined Print & E-Book Fiction
3619,LETHAL WHITE,The fourth book in the Cormoran Strike series....,by Robert Galbraith,Robert Galbraith,,0.00,,"Mulholland/Little, Brown",9780316422741,0316422746,5,1,2,0,0,2018-10-12,Combined Print & E-Book Fiction


In [30]:
#save to csv
combined_print_and_e_book_fiction.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2018to2023_combined_fiction.csv", index=False)

In [10]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "combined-print-and-e-book-fiction"   # you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "keys.json"

bestsellers = []
weeks = []
start_date = datetime.date(2018, 10, 5)  # Starting date
num_weeks = 260  # Number of weeks to fetch

for i in range(num_weeks):
    curr_date = start_date - datetime.timedelta(weeks=i)
    print(curr_date)
    url = "%s?list=%s&published-date=%s&api-key=%s" % (BOOKS_ROOT, LIST, curr_date, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    
    if r.status_code == 200:
        results = r.json()
        for result in results['results']:
            for book_details in result['book_details']:
                book_info = {
                    'title': book_details['title'],
                    'description': book_details['description'],
                    'contributor': book_details['contributor'],
                    'author': book_details['author'],
                    'contributor_note': book_details['contributor_note'],
                    'price': book_details['price'],
                    'age_group': book_details['age_group'],
                    'publisher': book_details['publisher'],
                    'primary_isbn13': book_details['primary_isbn13'],
                    'primary_isbn10': book_details['primary_isbn10'],
                    'rank': result['rank'],
                    'rank_last_week': result['rank_last_week'],
                    'weeks_on_list': result['weeks_on_list'],
                    'asterisk': result['asterisk'],
                    'dagger': result['dagger']
                }
                bestsellers.append(book_info)
                weeks.append(str(curr_date))
    else:
        print("API call did not work!")
        print(r)
    
    time.sleep(15)

2018-10-05
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2018-10-05&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-09-28
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2018-09-28&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-09-21
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2018-09-21&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-09-14
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2018-09-14&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-09-07
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2018-09-07&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-08-31
https://api.nytimes.com/svc/books/v3/lists.json?list=combined-print-and-e-book-fiction&published-date=2018-08-31&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-08-24

In [11]:
combined_print_and_e_book_fiction2 = pd.json_normalize(bestsellers)

In [12]:
combined_print_and_e_book_fiction2.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,LETHAL WHITE,The fourth book in the Cormoran Strike series....,by Robert Galbraith,Robert Galbraith,,0.00,,"Mulholland/Little, Brown",9780316422741,0316422746,1,0,1,0,0
1,TIME'S CONVERT,"During his lover's journey to immortality, a v...",by Deborah Harkness,Deborah Harkness,,0.00,,Penguin,9780399564529,None,2,0,1,0,0
2,ORIGIN,After reconnecting with one of his first stude...,by Dan Brown,Dan Brown,,0.00,,Doubleday,9780385542692,0385542690,3,11,30,0,0
3,JUROR #3,Ruby Bozarth defends a college football star c...,by James Patterson and Nancy Allen,James Patterson and Nancy Allen,,0.00,,"Little, Brown",9780316474122,0316474126,4,1,2,0,0
4,CRAZY RICH ASIANS,A New Yorker gets a surprise when she spends t...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780345803788,0345803787,5,3,15,0,0


In [13]:
combined_print_and_e_book_fiction2.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
4755,THE QUEST,Two journalists join an elderly priest who has...,by Nelson DeMille,Nelson DeMille,,0.00,,Center Street,9781455576425,1455576425,16,0,0,0,0
4756,DEADLINE,A journalist who has recently returned from Af...,by Sandra Brown,Sandra Brown,,0.00,,Grand Central,9781455501526,None,17,0,0,0,0
4757,THE CUCKOO'S CALLING,The first book in the Cormoran Strike series; ...,by Robert Galbraith,Robert Galbraith,,0.00,,"Mulholland/Little, Brown",9780316206860,0316206865,18,0,0,0,0
4758,THANKLESS IN DEATH,Lt. Eve Dallas searches for a young man who mu...,by J. D. Robb,J D Robb,,0.00,,Putnam,9781101617250,None,19,0,0,0,0
4759,THE FINAL CUT,"Nicholas Drummond, Scotland Yard’s new chief i...",by Catherine Coulter and J.T. Ellison,Catherine Coulter and JT Ellison,,0.00,,Putnam,9781101618493,,20,0,0,0,0


In [14]:
len(combined_print_and_e_book_fiction2)

4760

In [15]:
len(weeks)

4760

In [17]:
#add a week column
combined_print_and_e_book_fiction2['week'] = weeks

#and check to make sure it worked correctly
combined_print_and_e_book_fiction2.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
4740,GONE,"Detective Michael Bennett, living with his 10 ...",by James Patterson and Michael Ledwidge,James Patterson and Michael Ledwidge,,0.00,,"Little, Brown",9780316211017,031621101X,1,0,1,0,0,2013-10-18
4741,DOCTOR SLEEP,"Now grown up, Dan, the boy with psycho-intuiti...",by Stephen King,Stephen King,,0.00,,Scribner,9781476727660,147672766X,2,1,2,0,0,2013-10-18
4742,THE LONGEST RIDE,The lives of two couples converge unexpectedly.,by Nicholas Sparks,Nicholas Sparks,,0.00,,Grand Central,9781455520664,1455520667,3,2,3,0,0,2013-10-18
4743,SILENCING EVE,The final book of a trilogy about the forensic...,by Iris Johansen,Iris Johansen,,0.00,,St. Martin's,9781250020017,None,4,0,1,0,0,2013-10-18
4744,THE HUSBAND'S SECRET,A woman’s life is upended when she discovers a...,by Liane Moriarty,Liane Moriarty,,0.00,,Amy Einhorn/Putnam,9781101636237,,5,4,9,0,0,2013-10-18
4745,NEVER GO BACK,Jack Reacher is thrown into jail on trumped-up...,by Lee Child,Lee Child,,0.00,,Delacorte,9780440339373,0440339375,6,7,5,0,0,2013-10-18
4746,THE SIGNATURE OF ALL THINGS,A 19th-century botanist’s love of knowledge ta...,by Elizabeth Gilbert,Elizabeth Gilbert,,0.00,,Viking,9781101638002,None,7,0,1,0,0,2013-10-18
4747,THE RACKETEER,An imprisoned ex-lawyer schemes to exchange in...,by John Grisham,John Grisham,,0.00,,Doubleday,9780385536882,0385536887,8,11,20,0,0,2013-10-18
4748,W IS FOR WASTED,"A homeless man inexplicably leaves $600,000 to...",by Sue Grafton,Sue Grafton,,0.00,,Marian Wood/Putnam,9781101636459,1101636459,9,10,4,0,0,2013-10-18
4749,THE OCTOBER LIST,A woman innocently finds herself in possession...,by Jeffery Deaver,Jeffery Deaver,,0.00,,Grand Central,9781455576661,1455576662,10,0,1,0,0,2013-10-18


In [18]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
combined_print_and_e_book_fiction2['rank'] = combined_print_and_e_book_fiction2['rank'].astype(str)
filter_15 = combined_print_and_e_book_fiction2['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [20]:

combined_print_and_e_book_fiction2= combined_print_and_e_book_fiction2[filter_15]

In [21]:
len(combined_print_and_e_book_fiction2)

3900

In [22]:
#and a column indicating which list these come from
k2 = 3900
weekly_fiction_list = ['Combined Print & E-Book Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3900

In [23]:
#add nyt_list as a column
combined_print_and_e_book_fiction2['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
combined_print_and_e_book_fiction2.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
4730,THE CIRCLE,A dystopian novel about life on the campus of ...,by Dave Eggers,Dave Eggers,,0.00,,Knopf/McSweeney’s,9780385351409,0385351402,11,0,1,0,0,2013-10-25,Combined Print & E-Book Fiction
4731,WORTH THE FIGHT,A hard-bodied mixed martial arts fighter and a...,by Vi Keeland,Vi Keeland,,0.00,,Vi Keeland,A00B00FLG5B9S,None,12,0,1,0,0,2013-10-25,Combined Print & E-Book Fiction
4732,THE RACKETEER,An imprisoned ex-lawyer schemes to exchange in...,by John Grisham,John Grisham,,0.00,,Doubleday,9780385536882,0385536887,13,8,21,0,0,2013-10-25,Combined Print & E-Book Fiction
4733,THE SIGNATURE OF ALL THINGS,A 19th-century botanist’s love of knowledge ta...,by Elizabeth Gilbert,Elizabeth Gilbert,,0.00,,Viking,9781101638002,None,14,7,2,0,0,2013-10-25,Combined Print & E-Book Fiction
4734,CARTER REED,A woman who kills her roommate’s Mafia-connect...,by Tijan,Tijan,,0.00,,Tijan,A00B00FIOBPC0,None,15,0,1,0,0,2013-10-25,Combined Print & E-Book Fiction
4740,GONE,"Detective Michael Bennett, living with his 10 ...",by James Patterson and Michael Ledwidge,James Patterson and Michael Ledwidge,,0.00,,"Little, Brown",9780316211017,031621101X,1,0,1,0,0,2013-10-18,Combined Print & E-Book Fiction
4741,DOCTOR SLEEP,"Now grown up, Dan, the boy with psycho-intuiti...",by Stephen King,Stephen King,,0.00,,Scribner,9781476727660,147672766X,2,1,2,0,0,2013-10-18,Combined Print & E-Book Fiction
4742,THE LONGEST RIDE,The lives of two couples converge unexpectedly.,by Nicholas Sparks,Nicholas Sparks,,0.00,,Grand Central,9781455520664,1455520667,3,2,3,0,0,2013-10-18,Combined Print & E-Book Fiction
4743,SILENCING EVE,The final book of a trilogy about the forensic...,by Iris Johansen,Iris Johansen,,0.00,,St. Martin's,9781250020017,None,4,0,1,0,0,2013-10-18,Combined Print & E-Book Fiction
4744,THE HUSBAND'S SECRET,A woman’s life is upended when she discovers a...,by Liane Moriarty,Liane Moriarty,,0.00,,Amy Einhorn/Putnam,9781101636237,,5,4,9,0,0,2013-10-18,Combined Print & E-Book Fiction


In [24]:
#save to csv
combined_print_and_e_book_fiction2.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2013to2018_combined_fiction.csv", index=False)